In [2]:
from datetime import datetime
start_time = datetime.now()

# Libraries to install
# %pip install pandas-profiling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:01.134384


In [12]:
%%time
data = pd.read_csv('NY property data.csv')

CPU times: user 3.46 s, sys: 624 ms, total: 4.09 s
Wall time: 4.46 s


In [13]:
data.shape

(1070994, 32)

**Project 1 Variables**

We look at FULLVAL (full market value), AVLAND (Actual land value), AVTOT (Actual total value) from the dataset.

To standardize we create 3 sizes to calculate the ratio variables, and use them to create 3 normalized ratios for each:
* land/lot area = LTFRONT* LTDEPTH
* building area = BLDFRONT* BLDDEPTH
* building volume = (BLDFRONT* BLDDEPTH)*STORIES

Now we have 9 ratios for each record.

Then group by ZIP, ZIP3, TAXCLASS, BORO, in each group, calculate average ratio and divide the 9 ratios. Then we have  9 ratios for each group, in total 36 additional variables for each record, representing the particular property's primary variables $R_1$ to $R_9$ compared to their geographical(ZIP, BORO) or TAXCLASS neighbors

**Step 1**

Remove the exclusion records
* Remove government, public-owned property that we are not interested in. We are looking for private owner committing tax fraud.


In [14]:
#first look at the top owners
data['OWNER'].value_counts().head(20)

PARKCHESTER PRESERVAT    6021
PARKS AND RECREATION     4255
DCAS                     2169
HOUSING PRESERVATION     1904
CITY OF NEW YORK         1450
DEPT OF ENVIRONMENTAL    1166
BOARD OF EDUCATION       1015
NEW YORK CITY HOUSING    1014
CNY/NYCTA                 975
NYC HOUSING PARTNERSH     747
YORKVILLE TOWERS ASSO     558
DEPARTMENT OF BUSINES     527
DEPT OF TRANSPORTATIO     503
MTA/LIRR                  467
PARCKHESTER PRESERVAT     439
MH RESIDENTIAL 1, LLC     411
434 M LLC                 393
LINCOLN PLAZA ASSOCIA     366
DEUTSCHE BANK NATIONA     336
561 11TH AVENUE TMG L     324
Name: OWNER, dtype: int64

In [15]:
#create a remove list
remove_list = data['OWNER'].value_counts().head(20).index.tolist()
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [16]:
#some of them here are not government and we can also add some other to the list for our convenience
remove_list.remove('YORKVILLE TOWERS ASSO')
remove_list.remove('434 M LLC')
remove_list.remove('DEUTSCHE BANK NATIONA')
remove_list.remove('561 11TH AVENUE TMG L')
remove_list.append('UNITED STATES OF AMER')
remove_list.append('U S GOVERNMENT OWNRD')
remove_list.append('THE CITY OF NEW YORK')
remove_list.append('NYS URBAN DEVELOPMENT')
remove_list.append('NYS DEPT OF ENVIRONME')
remove_list.append('CULTURAL AFFAIRS')
remove_list.append('DEPT OF GENERAL SERVI')
remove_list.append('DEPT RE-CITY OF NY')

remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 'LINCOLN PLAZA ASSOCIA',
 'UNITED STATES OF AMER',
 'U S GOVERNMENT OWNRD',
 'THE CITY OF NEW YORK',
 'NYS URBAN DEVELOPMENT',
 'NYS DEPT OF ENVIRONME',
 'CULTURAL AFFAIRS',
 'DEPT OF GENERAL SERVI',
 'DEPT RE-CITY OF NY']

In [18]:
property_data = data[-data['OWNER'].isin(remove_list)].reset_index(drop=True)
property_data.shape

(1046826, 32)

In [19]:
#how much records we removed
len(data)-len(property_data)

24168

**Step 2**

Clean the data fields we are interested in and fill the missing value
* Remove null or meaningless value in fields of ZIP, STORIES, FULLVAL, AVLAND, AVTOT, LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH.
* We try to fill in missing fields with most typical value for the field in that record.
* Look for another related field, group by the field and calculate the average or mode of the missing field.

In [30]:
#filling in missing zip

#how many zips missing? replace NnaN with 0 and count
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
#we get the index list
missing_zips

array([    416,    2188,    2189, ..., 1046254, 1046442, 1046443])

In [31]:
len(missing_zips)

10245

In [32]:
#assume the data is sorted by zip, if a zip is missing and the before and after zip are same
#fill in the zip with that value
#we use loc to access the data with the index we have and locate it to specific column
for i in range(len(missing_zips)):
    if(property_data.loc[missing_zips[i]+1,'ZIP']==property_data.loc[missing_zips[i]-1,'ZIP']):
        property_data.loc[missing_zips[i],'ZIP']=property_data.loc[missing_zips[i]-1,'ZIP']

In [33]:
#how many NA still left to fill in
missing_zips= np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

In [34]:
#to fill the remaining zips we fill it with previous's record zip 
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i],'ZIP']=property_data.loc[missing_zips[i]-1,'ZIP']

In [35]:
missing_zips= np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

In [37]:
#pull records missing STORIES 
temp = property_data[property_data['STORIES'].isnull()]
len(temp)

43968

In [40]:
#some building have 1.5 stories
data['STORIES'].value_counts()

2.0      415092
3.0      130127
1.0       96706
2.5       82292
4.0       38342
          ...  
82.0          1
119.0         1
76.0          1
61.0          1
114.0         1
Name: STORIES, Length: 111, dtype: int64

In [39]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [46]:
#group records without stories by taxclass to find the common stories of that class
mean_stories = property_data.groupby('TAXCLASS')['STORIES'].mean()
mean_stories

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64

In [42]:
# no records have 0 for this field
len(property_data[property_data['STORIES']==0])

0

In [47]:
property_data['STORIES']=property_data['STORIES'].fillna(value=0)
for index in mean_stories.index:
    property_data.loc[(property_data['STORIES']==0)&(property_data['TAXCLASS']==index),'STORIES']=mean_stories[index]

In [49]:
#same for FULLVAL, AVLAND, AVTOT, we fill the average by TAXCLASS
#calculate means for AVTOT, FULLVAL, AVLAND by TAXCLASS and avoid record with 0

property_data['FULLVAL'].replace('NaN',0)
temp = property_data[property_data['FULLVAL']!=0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()

property_data['AVLAND'].replace('NaN',0)
temp = property_data[property_data['AVLAND']!=0]
mean_avland = temp.groupby('TAXCLASS')['AVLAND'].mean()

property_data['AVTOT'].replace('NaN',0)
temp = property_data[property_data['AVTOT']!=0]
mean_avtot = temp.groupby('TAXCLASS')['AVTOT'].mean()

In [50]:
for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL']==0)&(property_data['TAXCLASS']==index),'FULLVAL']=mean_fullval[index]
    property_data.loc[(property_data['AVLAND']==0)&(property_data['TAXCLASS']==index),'AVLAND']=mean_avland[index]
    property_data.loc[(property_data['AVTOT']==0)&(property_data['TAXCLASS']==index),'AVTOT']=mean_avtot[index]

In [54]:
#a different story for LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH
#notice from the DQR, many records are 0 and 1. Since a lot or building cannot be actually 1 foot long in the size
# Because these 4 fields do not have NAs, we just need to replace 0s and the 1s. 
# We think zero and 1 are both invalid values for these fields, so replace them with NA.
# Calculate groupwise average. Replace 0 and 1's by NAs so they are not counted in calculating mean.

property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
property_data.loc[property_data['LTFRONT']==1,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==1,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==1,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan

In [55]:
# Calculate mean now (mean function ignores NAs but not 0s hence we converted 0 to NA)
mean_LTFRONT=property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH=property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT=property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH=property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()


In [56]:
# Now impute values with the average for that record's TAXCLASS
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull())&(property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]    
    property_data.loc[(property_data['LTDEPTH'].isnull())&(property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull())&(property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull())&(property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]


**Step 3**

Calculate the ratio with the fields we cleaned above

In [58]:
#copy the cleaned data
data=property_data.copy()
data.shape

(1046826, 32)

In [59]:
#convert ZIP to a string rather than a float
data['ZIP']=data['ZIP'].astype(str)

#we call the first 3 digits of zip ZIP3
data['zip3']=data['ZIP'].str[:3]

In [61]:
#calculate the size for normalizing
data['ltsize']=data['LTFRONT']*data['LTDEPTH']
data['bldsize']=data['BLDFRONT']*data['BLDDEPTH']
data['bldvol']=data['bldsize']*data['STORIES']

In [62]:
data['r1'] = data['FULLVAL']/data['ltsize']
data['r2'] = data['FULLVAL']/data['bldsize']
data['r3'] = data['FULLVAL']/data['bldvol']
data['r4'] = data['AVLAND']/data['ltsize']
data['r5'] = data['AVLAND']/data['bldsize']
data['r6'] = data['AVLAND']/data['bldvol']
data['r7'] = data['AVTOT']/data['ltsize']
data['r8'] = data['AVTOT']/data['bldsize']
data['r9'] = data['AVTOT']/data['bldvol']

In [63]:
ninevars = ['r1','r2','r3','r4','r5','r6','r7','r8','r9']

zip5_mean = data.groupby('ZIP')[ninevars].mean() #find the local mean of each ratio in full zipcode 
zip3_mean = data.groupby('zip3')[ninevars].mean() #find local mean of each ratio in 3-digit bigger zipcode range
taxclass_mean = data.groupby('TAXCLASS')[ninevars].mean()
borough_mean = data.groupby('BORO')[ninevars].mean()

data = data.join(zip5_mean, on='ZIP',rsuffix='_zip5')
data = data.join(zip3_mean, on='zip3',rsuffix='_zip3')
data = data.join(taxclass_mean, on='TAXCLASS',rsuffix='_taxclass')
data = data.join(borough_mean, on='BORO',rsuffix='_boro')

rsuffix = ['_zip5','_zip3','_taxclass','_boro']
for var in ninevars:
    for r in rsuffix:
        data[str(var)+r] = data[var]/data[str(var)+r] 

In [64]:
#drop all the columns we do not need for the analysis, aka we only keeps the last 45 ratio cols created
result_data = data.iloc[:, -45:]

In [65]:
stats = result_data.describe().transpose()
stats.to_excel('stats_on_vars.xlsx')